In this notebook we will index the french dump version of the wikipedia dataset, and use with the retrieval for our qa/system

In [3]:
import numpy as np
from haystack.utils import clean_wiki_text, convert_files_to_docs, fetch_archive_from_http, print_answers

In [4]:
from datasets import load_dataset

In [5]:
wiki_corpus = load_dataset("wikipedia", "20220301.fr", split="train")

WARNING - datasets.load -  Using the latest cached version of the module from /Users/es.py/.cache/huggingface/modules/datasets_modules/datasets/wikipedia/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559 (last modified on Wed Jul 13 11:01:44 2022) since it couldn't be found locally at wikipedia., or remotely on the Hugging Face Hub.
WARNING - datasets.builder -  Reusing dataset wikipedia (/Users/es.py/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559)


In [6]:
wiki_corpus = wiki_corpus.shuffle(seed=42)

WARNING - datasets.arrow_dataset -  Loading cached shuffled indices for dataset at /Users/es.py/.cache/huggingface/datasets/wikipedia/20220301.fr/2.0.0/aa542ed919df55cc5d3347f42dd4521d05ca68751f50dbc32bae2a7f1e167559/cache-a0a7d2df10490c15.arrow


In [5]:
sample_wiki = wiki_corpus.shard(100, index=0)

In [6]:
sample_doc_dict = sample_wiki[1]

In [7]:
sample_doc_dict

{'id': '12576878',
 'url': 'https://fr.wikipedia.org/wiki/Teinture%20naturelle',
 'title': 'Teinture naturelle',
 'text': "Une teinture naturelle est une matière colorante dérivée de plantes ou d'invertébrés. On parle de teinture quand il s'agit de colorer des textiles au moyen de produits solubles dans l'eau ; les minéraux donnent des colorants insolubles appelés pigments qui s'appliquent mal à cet usage.\n\nLa plupart des teintures naturelles proviennent de racines, de baies, d'écorce, de feuilles ou de bois de plantes tinctoriales ou bien de champignons ou de lichens.\n\nHistoire \n\nLe concept de teinture naturelle se développe par opposition aux teintures issues de la chimie organique dans la deuxième moitié du . Il oppose aux produits de l'industrie chimique, les teintures que des artisans peuvent extraire de plantes, de lichens, d'insectes. Auparavant, on ne considérait comme naturelles que les quelques substances utilisables sans transformation, tandis que la plupart des teintu

In [10]:
from haystack.schema import Document

In [11]:
docs = Document.from_dict(sample_doc_dict, field_map={"text": "content"})

#### Building the Elastic Search Index

In [12]:
from haystack.utils import clean_wiki_text, convert_files_to_docs

In [8]:
from haystack.errors import HaystackError
from haystack.schema import Document
from typing import List, Optional, Generator, Set, Union, Callable, Dict
from copy import deepcopy
from haystack.nodes import PreProcessor
import re

In [7]:
from gensim.utils import deaccent

def remove_accents(document):
    input_without_accent = deaccent(document)
    return input_without_accent

In [9]:
async def convert_wiki_article_to_docs(
    item: dict,
    clean_func: Optional[Callable] = None,
    split_paragraphs: bool = False,
) -> List[Document]:
    """
    item

    :param items: dict of items
    :param clean_func: a custom cleaning function that gets applied to each doc (input: str, output:str)
    :param split_paragraphs: split text in paragraphs.
    """
    documents = []
    processed = 0
    text = item.get("text")
    text = remove_accents(text)
    if clean_func:
        text = clean_func(text)
    if split_paragraphs:
        for para in text.split("\n"):
            if 200 <= len(para.strip()) <= 2000:
                # just pick the paragraph with length between 50 and 1500
                processed += 1
                doc = Document(content=para, meta={"title": item.get("title")}, id=item.get("id"))
                documents.append(doc)
            else:
                continue
    else:
        processed += 1
        doc = Document(content=text, meta={"title": item.get("title")}, id=item.get("id"))
        documents.append(doc)
    
    return documents

In [10]:
import asyncio

#### Saving the Document in the retriever

In [11]:
from tqdm import tqdm
from tqdm.asyncio import tqdm_asyncio
from haystack.document_stores import ElasticsearchDocumentStore


In [12]:
INDEX_NAME = 'fr-wikipedia'

In [22]:

document_store = ElasticsearchDocumentStore(index=INDEX_NAME, recreate_index=False, analyzer="french")

/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [14]:
%%script false --no-raise-error
all_docs = []
for i in range(0, 100):
    shard = wiki_corpus.shard(100, index=i)
    with tqdm(total=shard.shape[0]) as pbar:
        docs_in_shard = tqdm_asyncio.gather(*[convert_wiki_article_to_docs(item, clean_func=clean_wiki_text, split_paragraphs=True) for item in shard])
        all_docs.append(docs_in_shard)
    print("done with shard ", i)

In [15]:
from  functools import reduce
from operator import iconcat

In [18]:
%%script false --no-raise-error
with tqdm(total=len(all_docs)) as pbar:
     scan_results = await tqdm_asyncio.gather(*all_docs[90:100])

In [17]:
%%script false --no-raise-error
scan_results = reduce(iconcat, scan_results, [])
scan_results = reduce(iconcat, scan_results, [])

With our document indexted int the elastic searh we can search , use the piaf dataset which have question with answers without paragraph and leverage them.

In [67]:
from collections import deque

In [20]:
from elasticsearch.helpers import bulk, parallel_bulk, scan
def write_documents_parallel(
        self,
        documents: Union[List[dict], List[Document]],
        index: Optional[str] = None,
        batch_size: int = 10_000,
        duplicate_documents: Optional[str] = None,
        headers: Optional[Dict[str, str]] = None,
    ):
        """
        Indexes documents for later queries in Elasticsearch./ an update for the index that use parralell bluk

        Behaviour if a document with the same ID already exists in ElasticSearch:
        a) (Default) Throw Elastic's standard error message for duplicate IDs.
        b) If `self.update_existing_documents=True` for DocumentStore: Overwrite existing documents.
        (This is only relevant if you pass your own ID when initializing a `Document`.
        If don't set custom IDs for your Documents or just pass a list of dictionaries here,
        they will automatically get UUIDs assigned. See the `Document` class for details)

        :param documents: a list of Python dictionaries or a list of Haystack Document objects.
                          For documents as dictionaries, the format is {"content": "<the-actual-text>"}.
                          Optionally: Include meta data via {"content": "<the-actual-text>",
                          "meta":{"name": "<some-document-name>, "author": "somebody", ...}}
                          It can be used for filtering and is accessible in the responses of the Finder.
                          Advanced: If you are using your own Elasticsearch mapping, the key names in the dictionary
                          should be changed to what you have set for self.content_field and self.name_field.
        :param index: Elasticsearch index where the documents should be indexed. If not supplied, self.index will be used.
        :param batch_size: Number of documents that are passed to Elasticsearch's bulk function at a time.
        :param duplicate_documents: Handle duplicates document based on parameter options.
                                    Parameter options : ( 'skip','overwrite','fail')
                                    skip: Ignore the duplicates documents
                                    overwrite: Update any existing documents with the same ID when adding documents.
                                    fail: an error is raised if the document ID of the document being added already
                                    exists.
        :param headers: Custom HTTP headers to pass to elasticsearch client (e.g. {'Authorization': 'Basic YWRtaW46cm9vdA=='})
                Check out https://www.elastic.co/guide/en/elasticsearch/reference/current/http-clients.html for more information.
        :raises DuplicateDocumentError: Exception trigger on duplicate document
        :return: None
        """

        if index and not self.client.indices.exists(index=index, headers=headers):
            self._create_document_index(index, headers=headers)

        if index is None:
            index = self.index
        duplicate_documents = duplicate_documents or self.duplicate_documents
        assert (
            duplicate_documents in self.duplicate_documents_options
        ), f"duplicate_documents parameter must be {', '.join(self.duplicate_documents_options)}"

        field_map = self._create_document_field_map()
        document_objects = [Document.from_dict(d, field_map=field_map) if isinstance(d, dict) else d for d in documents]
        document_objects = self._handle_duplicate_documents(
            documents=document_objects, index=index, duplicate_documents=duplicate_documents, headers=headers
        )
        documents_to_index = []
        for doc in tqdm(document_objects):
            _doc = {
                "_op_type": "index" if duplicate_documents == "overwrite" else "create",
                "_index": index,
                **doc.to_dict(field_map=self._create_document_field_map()),
            }  # type: Dict[str, Any]

            # cast embedding type as ES cannot deal with np.array
            if _doc[self.embedding_field] is not None:
                if type(_doc[self.embedding_field]) == np.ndarray:
                    _doc[self.embedding_field] = _doc[self.embedding_field].tolist()

            # rename id for elastic
            _doc["_id"] = str(_doc.pop("id"))

            # don't index query score and empty fields
            _ = _doc.pop("score", None)
            _doc = {k: v for k, v in _doc.items() if v is not None}

            # In order to have a flat structure in elastic + similar behaviour to the other DocumentStores,
            # we "unnest" all value within "meta"
            if "meta" in _doc.keys():
                for k, v in _doc["meta"].items():
                    _doc[k] = v
                _doc.pop("meta")
            documents_to_index.append(_doc)

            # Pass batch_size number of documents to bulk
            if len(documents_to_index) % batch_size == 0:
                pb_ = parallel_bulk(self.client, 
                              documents_to_index, 
                              chunk_size=10000, 
                              thread_count=8, 
                              queue_size=8,
                              refresh=self.refresh_type, 
                              headers=headers)
                deque(pb_, maxlen=0)
                documents_to_index = []

        if documents_to_index:
            pb_= parallel_bulk(self.client, 
                          documents_to_index, 
                          chunk_size=10000, 
                          thread_count=8, 
                          queue_size=8,
                          refresh=self.refresh_type, 
                          headers=headers)
            deque(pb_, maxlen=0)

In [23]:
document_store.write_documents_parallel = write_documents_parallel

In [24]:
%%script false --no-raise-error
write_documents_parallel(document_store, scan_results)

In [25]:
from haystack.nodes import BM25Retriever

In [57]:
bm25_retriever = BM25Retriever(document_store=document_store, all_terms_must_match=False)

In [27]:
import pandas as pd

In [28]:
from pathlib import Path
DATA_PATH = Path.cwd().joinpath("data")
assert DATA_PATH.exists(), "the data path does not exist"

In [85]:
piaf_file = DATA_PATH.joinpath("corpus", "raw", "piaf", "questoin-reponse.csv")

In [86]:
assert piaf_file.exists(), "the piaf dataset does not exist"

piaf_question = data

In [87]:
piaf_df_without_context = pd.read_csv(piaf_file)

In [88]:
sample_question_response = piaf_df_without_context.sample(1)
question = deaccent(sample_question_response.question.values[0])
response = deaccent(sample_question_response.reponse.values[0])


In [89]:
question

'Quels plats de viandes epicees sont prepares a partir de ces ingredients ?'

In [29]:
def get_positive_context(retriever: BM25Retriever, search_query:str, answer:str, positive_documents: int = 100) -> List[Document]:    
        """given entitity retrieve the positive context
        we will first retrieve the top  100 documents , 
        - if the answer is in the top 40 document the input of the reader is the top 40 documents
        if the top 40 documents does not contain the answer we check whithin the top 41 to 100 document if the anwer is ther and we put it ther.
        other wise we discard the sentence

        Args:
            retriever (BM25Retriever): _description_
            n_ctxs (int, optional): _description_. Defaults to 15.
            entity (Entity, optional): _description_. Defaults to None.
        """
        list_pos_ctxs = []
        retrieved_docs = retriever.retrieve(query=search_query, top_k=positive_documents)
        for index, retrieve_doc in enumerate(retrieved_docs[0:40]):
            if answer.lower() in retrieve_doc.content.lower():
                list_pos_ctxs.append(
                    {"title": retrieve_doc.meta.get("title"), "content": retrieve_doc.content}
                )
        if len(list_pos_ctxs) == 0:
            for index, retrieve_doc in enumerate(retrieved_docs[40:100]):
                if answer.lower() in retrieve_doc.content.lower():
                    list_pos_ctxs.append(
                        {"title": retrieve_doc.meta.get("title"), "content": retrieve_doc.content}
                    )
        else:
            pass
        if len(list_pos_ctxs) == 0:
            return []
        return list_pos_ctxs

#### Use Piaf Dataset to query context

In [30]:
piaf_dataset = load_dataset("piaf")

WARNING - datasets.load -  Using the latest cached version of the module from /Users/es.py/.cache/huggingface/modules/datasets_modules/datasets/piaf/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae (last modified on Thu Jul 14 08:39:15 2022) since it couldn't be found locally at piaf., or remotely on the Hugging Face Hub.
WARNING - datasets.builder -  Reusing dataset piaf (/Users/es.py/.cache/huggingface/datasets/piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae)


  0%|          | 0/1 [00:00<?, ?it/s]

In [186]:
piaf_dataset = piaf_dataset["train"]

In [92]:
for index in tqdm(piaf_df_without_context.index):
    question = piaf_df_without_context.loc[index].question
    response = piaf_df_without_context.loc[index].reponse
    question = deaccent(question)
    response = deaccent(response)
    retrieved_docs = get_positive_context(retriever=bm25_retriever, search_query=question, answer=response, positive_documents=100)
    piaf_df_without_context.loc[index, "positive_context"] = retrieved_docs

/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


By using our wiki corpus we are able to find some question with positive context, we will leverage them to build our qa system

In [93]:
piaf_with_context = piaf_df_without_context.loc[piaf_df_without_context.positive_context.apply(lambda x: len(x)) > 0]

In [94]:
piaf_with_context.shape

(2459, 3)

with our dataset , we can find that {{piaf_with_context.shape[0]}} have positive context and that will be usefull to fine tune our model.

In [95]:
piaf_with_context = piaf_with_context.assign(length_positive_context =piaf_with_context["positive_context"].apply(lambda x: len(x)))

In [117]:
piaf_with_context.loc[piaf_with_context.length_positive_context > 1].sort_values(by="length_positive_context", ascending=False)

,question,reponse,positive_context,length_positive_context
6756,quel mot ne figure pas dans le webster?,A,"[{'title': 'They singulier', 'content': 'En , le pronom au singulier fait s...",40
2516,Un dirigeant a-t-il séparé sa région de l'empire ?,Le,"[{'title': 'Conquêtes mongoles', 'content': 'Durant ces annees, l'empire se ...",39
6762,quel est le nom court pour République italienne?,Italie,"[{'title': 'Constitution de l'Italie', 'content': 'La constitution italienne...",38
5539,Comment accordait-on le mridang?,on,"[{'title': 'Parchemin', 'content': ' tambours : djembe (chevre, antilope, ze...",38
11413,qui a parachuté des provisions et mitraillé les positions chinoises ?,la,"[{'title': 'Pierre Claude', 'content': 'Le , lors d'un combat aerien contre ...",37
...,...,...,...,...
5891,Dans quelle matières sont façonnés les bijoux algériens ?,argent,"[{'title': 'Bouton de manchette', 'content': 'Il est souvent considere comme...",2
5918,Combien d'écoles anglaises appartiennent aux 100 meilleures universités en 2...,12,"[{'title': 'Flávio Augusto da Silva', 'content': 'Ne et eleve dans la banlie...",2
5919,Combien d'écoles anglaises appartiennent aux 100 meilleures universités en 2...,12,"[{'title': 'Flávio Augusto da Silva', 'content': 'Ne et eleve dans la banlie...",2
5920,où est classée l'université d'oxford en 2018 ?,première,"[{'title': 'Alfred Dürr', 'content': 'Durr etudia la musicologie et la philo...",2


In [33]:
retrieved_docs = bm25_retriever.retrieve(query=
    deaccent("De quelle langue est issue le mot mycelium ?"), top_k=30)

Retriever query: {'size': '30', 'query': {'bool': {'must': [{'multi_match': {'query': 'De quelle langue est issue le mot mycelium ?', 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}


In [32]:
retrieved_docs

[<Document: {'content': 'Un mot savant est un neologisme ou un mot dont l’evolution etymologique a ete freinee par le contexte socio-historique et qui reste de ce fait plus proche de son etymon. Par exemple, « fragile » est un mot savant dont le correspondant populaire est « frele ». Plus generalement, la langue savante (marquee de formations savantes) est la langue scientifique issue du latin ou du grec, opposee a la langue populaire ou vulgaire.', 'content_type': 'text', 'score': 0.9178798627418258, 'meta': {'title': 'Mot savant'}, 'embedding': None, 'id': '9713327'}>,
 <Document: {'content': "La croissance du mycelium qui a lieu dans l'espace intercellulaire, est fortement correlee a la croissance de l'hote. Le mycelium colonise les nouvelles feuilles et les nouveaux talles. La transmission par la semence est tres efficace. Dans la graine le mycelium est localise essentiellement dans la couche a aleurone.", 'content_type': 'text', 'score': 0.8908335557594226, 'meta': {'title': 'Epic

In [118]:
piaf_with_context.to_csv(DATA_PATH.joinpath("corpus", "raw", "piaf", "piaf_with_context.csv"))

within our dataset , 740 row have more than one context.

With our dataset with context, let us pull more question with context for the original piaf dataset with context

In [119]:
piaf_dataset = load_dataset("piaf")

WARNING - datasets.load -  Using the latest cached version of the module from /Users/es.py/.cache/huggingface/modules/datasets_modules/datasets/piaf/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae (last modified on Thu Jul 14 08:39:15 2022) since it couldn't be found locally at piaf., or remotely on the Hugging Face Hub.
WARNING - datasets.builder -  Reusing dataset piaf (/Users/es.py/.cache/huggingface/datasets/piaf/plain_text/1.0.0/535c60f4155fe1d644c1746e86131963c082f309d0dbb5ba4d606786c7f4a6ae)


  0%|          | 0/1 [00:00<?, ?it/s]

In [38]:
piaf_dataset = piaf_dataset["train"]

In [39]:
piaf_df = piaf_dataset.to_pandas()

In [40]:
piaf_df.head()

,id,title,context,question,answers
0,p140295443291664,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Combien de personnes travaillent au ministère des sports,"{'text': ['100 000'], 'answer_start': [472]}"
1,p140295443291520,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Combien d'employeurs,"{'text': ['20 000'], 'answer_start': [597]}"
2,p140295443291376,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel part du budget des ménages,"{'text': ['50'], 'answer_start': [46]}"
3,p140295443291088,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel montant en 2003,"{'text': ['14,2 milliards'], 'answer_start': [68]}"
4,p140295443290872,Sport,"Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 mil...",Quel montant en 2019,"{'text': ['12 milliards'], 'answer_start': [102]}"


What are we trying to achive with this ?


We have question on the piaf dataset with context, for those question we will query elastic search to find out  the additional context. That context will will be consider as additional context to train our model with.


The algo : 

- for each question , query the elastic search and keep the top 10 documents context
- we will loop and send queries in a batch of 10 questions to elastic search
- retrieved the context and and save anything on the disk.

In the future to improve the quality of our finding we can consider only documents with named entities in the answers. We can check the paragraph , run the NER model on it and then check if the answer is in the named entities.

In [41]:
sample_questions = np.vectorize(deaccent)(piaf_df.question.loc[1:5])
retrieved_docs = bm25_retriever.retrieve_batch(queries=sample_questions.tolist(), top_k=30)

/Users/es.py/Projects/Personal/unsupervised-open-domain-french-question-answering/.venv/lib/python3.9/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.16/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)


In [35]:
async def query_batch(retriever, queries, top_k=10):
    return retriever.retrieve_batch(queries=queries, top_k=top_k)

In [36]:
def decent_vectorize(queries):
    return np.vectorize(deaccent)(queries)

In [42]:
question_chunk = [decent_vectorize(piaf_df.question.loc[1:5]), decent_vectorize(piaf_df.question.loc[5:10])]

In [43]:
question_chunk

[array(["Combien d'employeurs", 'Quel part du budget des menages',
        'Quel montant en 2003', 'Quel montant en 2019',
        'En quelle annee Jakob Bohme tombe-t-il malade ?'], dtype='<U47'),
 array(['En quelle annee Jakob Bohme tombe-t-il malade ?',
        'Qui est mort en juillet ?',
        'Quel est le metier de Nicolas Thomas ?',
        'Que doit subir Jakob Bohme avant d’obtenir les derniers sacrements ?',
        'Pourquoi Bohme enfle-t-il ?',
        'Quel pays est surnomme la "perle de l\'Afrique" ?'], dtype='<U68')]

In [244]:
async def main():
    return await tqdm_asyncio.gather(*[query_batch(bm25_retriever, queries) for queries in question_chunk])

In [44]:
def write_to_json(data, path):
    with open(path, "w") as f:
        json.dump(data, f, indent=4, ensure_ascii=False)

In [45]:
question_chunk[0][-1]

'En quelle annee Jakob Bohme tombe-t-il malade ?'

In [47]:
answers[0][-1]

NameError: name 'answers' is not defined

In [48]:
piaf_dataset[0]

{'id': 'p140295443291664',
 'title': 'Sport',
 'context': "Les dépenses des ménages représentent plus de 50 % de ces montants (14,2 milliards d'euros en 2003 et 12 milliards d'euros en 2019), contre 7,9 milliards d'euros pour les collectivités locales, 3,2 pour l'État, et 2,2 pour les entreprises. Parmi les dépenses sportives des ménages en 2003, 3,7 milliards sont consacrés aux vêtements de sport et chaussures, 2 aux biens durables, 2,7 aux autres biens et 5,8 aux services. Le Ministère de la Jeunesse et des Sports estime à 100 000 (58 % d'hommes pour 42 % de femmes) le nombre de salariés travaillant pour le secteur sportif en France pour quelque 20 000 employeurs.",
 'question': 'Combien de personnes travaillent au ministère des sports',
 'answers': {'text': ['100 000'], 'answer_start': [472]}}

In [49]:
import json

In [50]:
async def process_queries_chunk(retriever:BM25Retriever, queries_chunk: pd.DataFrame):
    """tak a query chunk process it , query the elastic, instance and write the document to the file.

    Args:
        queries_chunk (pd.DataFrame): _description_
    """
    questions = decent_vectorize(queries_chunk.question)
    titles = queries_chunk.title
    answers = [answer.get("text")[0] for answer in queries_chunk.answers]
    contexts = queries_chunk.context
    ids = queries_chunk.id
    retrieved_docs = retriever.retrieve_batch(queries=questions, top_k=10)
    process_answers(ids=ids, 
                    questions=questions,
                    titles=titles,
                    answers=answers,
                    contexts=contexts,
                    retrieved_docs=retrieved_docs)

In [51]:
def process_doc(retrieved_docs):
    """
    convert a list of retrieved document to list of array of list and content
    """
    doc_list = []
    for doc in retrieved_docs:
        doc_ = {
            "title": doc.meta.get("title"),
            "content": doc.content,
            }
        doc_list.append(doc_)
    return doc_list

In [52]:
piaf_with_multi_context_path = DATA_PATH.joinpath("corpus", "french-qa", "piaf-with-multi-context")
assert piaf_with_multi_context_path.exists()

In [101]:
def process_answers(ids, questions, titles, answers, contexts, retrieved_docs):
    """process answers and write them to the file.

    Args:
        questions (_type_): _description_
        titles (_type_): _description_
        answers (_type_): _description_
        contexts (_type_): _description_
        retrieved_docs (_type_): _description_
    """
    for id_, question, title, answer, context, retrieved_doc in zip(ids, questions, titles, answers, contexts, retrieved_docs):
        contexts = [{"title": title, "content": context}]
        additional_context = process_doc(retrieved_doc)
        contexts.extend(additional_context)
        instance_json = {
            "question": question,
            "answer": answer,
            "contexts": contexts,
            "id": id_,
        }
        yield instance_json

    

In [102]:
def save_to_json(instances, path):
    "saves an iterator of multiple json files to files in the path directory"
    for instance in instances:
        with open(path.joinpath(f"{instance['id']}.json"), "w") as f:
            json.dump(instance, f, indent=4, ensure_ascii=False)

In [54]:
async def main():
    return await tqdm_asyncio.gather(*[process_queries_chunk(bm25_retriever, queries) for _, queries in piaf_df.groupby(np.arange(len(piaf_df))//5)])

In [55]:
%%script false --no-raise-error
await main()

In [58]:
bm25_retriever.retrieve(query=
    deaccent("De quelle langue est issue le mot mycelium ?"), top_k=10)

Retriever query: {'size': '10', 'query': {'bool': {'must': [{'multi_match': {'query': 'De quelle langue est issue le mot mycelium ?', 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}


[<Document: {'content': 'Un mot savant est un neologisme ou un mot dont l’evolution etymologique a ete freinee par le contexte socio-historique et qui reste de ce fait plus proche de son etymon. Par exemple, « fragile » est un mot savant dont le correspondant populaire est « frele ». Plus generalement, la langue savante (marquee de formations savantes) est la langue scientifique issue du latin ou du grec, opposee a la langue populaire ou vulgaire.', 'content_type': 'text', 'score': 0.9178798627418258, 'meta': {'title': 'Mot savant'}, 'embedding': None, 'id': '9713327'}>,
 <Document: {'content': "La croissance du mycelium qui a lieu dans l'espace intercellulaire, est fortement correlee a la croissance de l'hote. Le mycelium colonise les nouvelles feuilles et les nouveaux talles. La transmission par la semence est tres efficace. Dans la graine le mycelium est localise essentiellement dans la couche a aleurone.", 'content_type': 'text', 'score': 0.8908335557594226, 'meta': {'title': 'Epic

##### Processing Fquad

In [59]:
frquad_path_train = DATA_PATH.joinpath("corpus", "french-qa", "fquad", "train.json")
frquad_path_valid = DATA_PATH.joinpath("corpus", "french-qa", "fquad", "valid.json")

In [60]:
assert frquad_path_train.exists()
assert frquad_path_valid.exists()

In [61]:
with open(frquad_path_train, "r") as f:
    frquad_train = json.load(f)

In [63]:
frquad_train = frquad_train.get("data")

In [147]:
with open(frquad_path_valid, "r") as f:
    frquad_valid = json.load(f)

In [149]:
frquad_valid = frquad_valid.get("data")

In [151]:
frquad_valid[0]

{'title': 'anges-musiciens-(national-gallery)',
 'paragraphs': [{'qas': [{'answers': [{'answer_start': 161,
       'text': 'La Vierge aux rochers'}],
     'id': 'a524504f-0816-4f58-9f2d-27f82a85c73d',
     'question': 'Que concerne principalement les documents ?'},
    {'answers': [{'answer_start': 46, 'text': 'documents contemporains'}],
     'id': '8a72ad1c-b2fe-4fe6-9f87-35fcb713cf38',
     'question': 'Par quoi sont décrit les deux tableaux ?'},
    {'answers': [{'answer_start': 204, 'text': 'objets de spéculations'}],
     'id': 'b2db7f77-f6a7-4c3d-9274-9807f9764e97',
     'question': "Quels types d'objets sont les deux tableaux aux yeux des chercheurs ?"}],
   'context': "Les deux tableaux sont certes décrits par des documents contemporains à leur création mais ceux-ci ne le font qu'indirectement car ils concernent principalement La Vierge aux rochers. Aussi demeurent-ils objets de spéculations pour les chercheurs quant à leur statut de première ou seconde version de l'œuvre, leu

In [98]:
from itertools import repeat

In [107]:
fquad_path = DATA_PATH.joinpath("corpus", "french-qa", "fquad-with-multi-context")
fquad_path_output_train = fquad_path.joinpath("train")
fquad_path_output_valid = fquad_path.joinpath("valid")

In [136]:
for doc in frquad_train[0:2]:
    print(doc.get("title"))

(1)-cérès
american-idiot


In [115]:
async def process_fquad_paragraph(retriever, paragraph, title, path):
    """
    this will loop over the  a paragraph in frquad dataset.
    since each paragraph have a list of question ,
     it will return query the elasticsearch for each batch of question 
    and return the additional related_context

    Args:
        paragraph (_type_): _description_
    """
    context  = paragraph.get("context")
    question_answers = paragraph.get("qas")
    ids = [qa.get("id") for qa in question_answers]
    questions = [qa.get("question") for qa in question_answers]
    answers = [qa.get("answers")[0].get("text") for qa in question_answers]
    retrieved_docs = retriever.retrieve_batch(queries=questions, top_k=10)
    instances = process_answers(ids=ids, 
                                questions=questions,
                                titles=repeat(title, len(questions)),
                                answers=answers,
                                contexts=repeat(context, len(questions)),
                                retrieved_docs=retrieved_docs)
    save_to_json(instances, path)

In [106]:
assert fquad_path.exists()

In [79]:
def check_answer_in_retrieved_docs(answer, retrieved_docs):
    for doc in retrieved_docs:
        if answer in doc.content:
            print(doc)
            return True
    return False

In [86]:
retrieved_docs = bm25_retriever.retrieve(query=deaccent("Combien de fois Piazzi est-il parvenu à observer Cérès?"), top_k=30)


Retriever query: {'size': '30', 'query': {'bool': {'must': [{'multi_match': {'query': 'Combien de fois Piazzi est-il parvenu a observer Ceres?', 'type': 'most_fields', 'fields': ['content'], 'operator': 'OR'}}]}}}


In [119]:
await process_fquad_paragraph(bm25_retriever, frquad_train[0]["paragraphs"][0], "Cérès", fquad_path_output_train)

In [139]:
async def process_fquad(fquad, path, retriever):
    all_article_processor = list()
    for document in fquad:
        title = document.get("title")
        paragraphs = document.get("paragraphs")
        doc_coroutine = tqdm_asyncio.gather(*[process_fquad_paragraph(retriever=bm25_retriever, paragraph=paragraph, title=title, path=path) for paragraph in paragraphs])
        all_article_processor.append(doc_coroutine)
    return await tqdm_asyncio.gather(*all_article_processor)

In [141]:
await process_fquad(frquad_train, fquad_path_output_train, bm25_retriever)

  0%|          | 0/117 [00:00<?, ?it/s]





























































































































































































100%|██████████| 37/37 [02:02<00:00,  3.32s/it]  [A

100%|██████████| 70/70 [02:02<00:00,  1.75s/it]  


100%|██████████| 1/1 [02:02<00:00, 122.82s/it]



100%|██████████| 33/33 [02:02<00:00,  3.72s/it]  




100%|██████████| 39/39 [02:02<00:00,  3.15s/it]  





100%|██████████| 94/94 [02:02<00:00,  1.31s/it]  






100%|██████████| 30/30 [02:02<00:00,  4.09s/it]







100%|██████████| 9/9 [02:02<00:00, 13.64s/it] 








100%|██████████| 53/53 [02:02<00:00,  2.32s/it]  









100%|██████████| 32/32 [02:02<00:00,  3.84s/it]  










100%|██████████| 27/27 [02:02<00:00,  4.55s/it]











100%|██████████| 11/11 [02:02<00:00, 11.17s/it]












100%|██████████| 90/90 [02:02<00:00,  1.37s/it]  













100%|██████████| 17/17 [02:02<00:00,  7.

[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  Non

In [153]:
await process_fquad(frquad_valid, fquad_path_output_valid, bm25_retriever)






































100%|██████████| 18/18 [00:23<00:00,  1.32s/it]


[[None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None

At this point we have the piaf dataset with context, we have the frquad with context, let us look now the exetat questions with context.

In [173]:
exetat_questions_path = DATA_PATH.joinpath("corpus", "french-qa", "exetat-questions", "questions.json")

In [174]:
assert exetat_questions_path.exists()

In [192]:
with open(exetat_questions_path, "r") as f:
    exetat_questions = json.load(f)

In [195]:
len(exetat_questions)

173

for now we can say that we have 173 exams questions, we need more and get more context to train.frquad_train
But for now we are going to train our model on the conbinaision of piaf and frenchquad dataset.